Para realizar a análise, regressão e classificação, escolhemos o dataset Housing Prices, disponível em:

https://www.kaggle.com/datasets/yasserh/housing-prices-dataset

O dataset é composto por apenas 1 arquivo .csv separado por vírgulas (",")
O arquivo contém 13 colunas, sendo o objetivo realizar a regressão para inferir o preço das casas e realizar a classificação para inferir quais casas possuem sala de visitas.

1) importar as dependências necessárias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

2) leitura do arquivo

In [ ]:
df = pd.read_csv('Housing.csv', delimiter=',', header=0)
df.head()

3) limpeza de dados indesejados

In [ ]:
df_cp = df.copy()
obj_cols = [col for col in df_cp.columns if df_cp[col].dtype == 'object']
# para colunas de valor booleano (sim ou não) devem existir apenas 2 tipos de resposta, furnishingstatus deve possuir 3 valores
print(f"Valores repetidos:\n{df[obj_cols].nunique()}")

# caso exista valores fora do padrão, devem ser substituidos
valores_aceitos = ['yes', 'no']
valores_furnishingstatus = ['furnished', 'semi-furnished', 'unfurnished']

for col in df_cp.columns:
    if col in obj_cols:
        if col == 'furnishingstatus':
            df_cp[col] = df_cp[col].apply(lambda x: x if x in valores_furnishingstatus else 'unfurnished')
        else:
            df_cp[col] = df_cp[col].apply(lambda x: x if x in valores_aceitos else 'no')
    else:
        df_cp[col] = df_cp[col].fillna(df[col].median())


print(f"\n\nValores faltantes:\n{df_cp.isnull().sum()}")

4) Transformação dos dados de atributos qualitativos para quantitativos e análise descritiva

In [ ]:
# transformação de dados qualitativos em quantitativos
le = LabelEncoder()
for col in obj_cols:
  df_cp[col] = le.fit_transform(df_cp[col])


5) mostrar a correlação entre colunas

In [ ]:
# plot do mapa de calor de correlação
corr_person = df_cp.corr(method='pearson')
corr_sperman = df_cp.corr(method='spearman')
corr_kendall = df_cp.corr(method='kendall')
corr_list = [corr_person, corr_sperman, corr_kendall]
for corr in corr_list:
  plt.figure(figsize=(8, 6))
  sns.heatmap(corr, cmap="YlGnBu", annot=True, fmt=".1f")

É possível verificar que Price possui índices de correlação rasoáveis e Guestroom possui índices de correlação baixos.

No geral, isso pode implicar em melhor acurácia para regressão do campo Price e pior acurácia para a classificação do campo Guestroom

Para a redução de dimensionalidade, é escolhido remover os campos com menor correlação para as colunas alvo.


In [ ]:
del df_cp['mainroad']
del df_cp['hotwaterheating']
del df_cp['prefarea']
del df_cp['furnishingstatus']

4) divisão do dataset em conjunto de treinamento e conjunto de teste

In [ ]:
training = df_cp.sample(frac=0.8)
test = df_cp.drop(training.index)

training_price = training['price']
training_guestroom = training['guestroom']

test_price = test['price']
test_guestroom = test['guestroom']

del training['price']
del training['guestroom']

del test['price']
del test['guestroom']